Name: Wenhao Wu

ID: 1304952

In [38]:
#ref https://livebook.manning.com/#!/book/deep-learning-with-python/chapter-3/316

import numpy as np
import keras
import matplotlib.pyplot as plt

In [41]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

mean = np.mean(train_data, axis=0)
std = np.std(train_targets, axis=0)

train_data -= mean
train_data /= (2*std)

test_data -= mean
test_data /= (2*std)

The above cell is to normalize the input, we should somehow shuffle the data first.

In [34]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='sgd', loss='mse', metrics=['mae'])
    return model

We define a method to init the model, since in k-fold validation we need to init a clean model multiple times

In [43]:
k = 4
num_val_samples = len(train_data)
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #',i)
    val_data = traind_data[i * num_val_samples: (i+1) * ]

404
processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
